In [2]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import boxscoreadvancedv3, teamgamelog


/Users/akaman150/Desktop/nba_predictor_development/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Example: Fetching all games for a particular season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2018-19')
games = gamefinder.get_data_frames()[0]
game_data = games[['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']].copy(deep=True)
home_team_data = game_data[game_data['MATCHUP'].str.contains('vs.')].copy(deep=True)
away_team_data = game_data[game_data['MATCHUP'].str.contains('@')].copy(deep=True)
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42018,1610612744,GSW,Golden State Warriors,0041800406,2019-06-13,GSW vs. TOR,L,240,110,...,0.700,11,31,42,28,9,6,16,23,-4.0
1,42018,1610612761,TOR,Toronto Raptors,0041800406,2019-06-13,TOR @ GSW,W,241,114,...,0.793,11,28,39,25,8,2,12,23,4.0
2,42018,1610612744,GSW,Golden State Warriors,0041800405,2019-06-10,GSW @ TOR,W,240,106,...,0.714,6,31,37,27,5,7,15,22,1.0
3,42018,1610612761,TOR,Toronto Raptors,0041800405,2019-06-10,TOR vs. GSW,L,238,105,...,0.778,13,30,43,19,6,5,13,19,-1.0
4,42018,1610612744,GSW,Golden State Warriors,0041800404,2019-06-07,GSW vs. TOR,L,241,92,...,0.667,8,34,42,26,6,6,17,23,-13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4165,12018,1610612742,DAL,Dallas Mavericks,0011800004,2018-09-29,DAL vs. BJD,W,241,116,...,0.639,12,39,51,31,6,10,13,29,53.0
4166,12018,1610612755,PHI,Philadelphia 76ers,0011800001,2018-09-28,PHI vs. MEL,W,239,104,...,0.640,11,37,48,25,13,9,17,19,20.0
4167,12018,15016,MEL,Melbourne United,0011800001,2018-09-28,MEL @ PHI,L,238,84,...,0.929,8,29,37,15,10,1,20,26,-20.0
4168,12018,1610612766,CHA,Charlotte Hornets,0011800002,2018-09-28,CHA vs. BOS,W,241,104,...,0.641,12,53,65,17,4,7,17,22,7.0


In [4]:
home_team_data.rename(columns={'TEAM_ABBREVIATION': 'H_TEAM', "WL" : "H_TEAM_WIN"}, inplace=True)
# encode win/loss as 1/0
# home_team_data['H_TEAM_WIN'] = home_team_data['H_TEAM_WIN'].replace('W', 1)
# home_team_data['H_TEAM_WIN'] = home_team_data['H_TEAM_WIN'].replace('L', 0)

home_team_data

,SEASON_ID,TEAM_ID,H_TEAM,GAME_ID,GAME_DATE,MATCHUP,H_TEAM_WIN
0,42018,1610612744,GSW,0041800406,2019-06-13,GSW vs. TOR,L
3,42018,1610612761,TOR,0041800405,2019-06-10,TOR vs. GSW,L
4,42018,1610612744,GSW,0041800404,2019-06-07,GSW vs. TOR,L
7,42018,1610612744,GSW,0041800403,2019-06-05,GSW vs. TOR,L
8,42018,1610612761,TOR,0041800402,2019-06-02,TOR vs. GSW,L
...,...,...,...,...,...,...,...
4162,12018,1610612744,GSW,0011800005,2018-09-29,GSW vs. MIN,L
4164,12018,1610612761,TOR,0011800003,2018-09-29,TOR vs. POR,W
4165,12018,1610612742,DAL,0011800004,2018-09-29,DAL vs. BJD,W
4166,12018,1610612755,PHI,0011800001,2018-09-28,PHI vs. MEL,W


In [5]:
away_team_data.rename(columns={'TEAM_ABBREVIATION': 'A_TEAM'}, inplace=True)
# away_team_data.drop(columns=['WL'], inplace=True)
away_team_data

,SEASON_ID,TEAM_ID,A_TEAM,GAME_ID,GAME_DATE,MATCHUP,WL
1,42018,1610612761,TOR,0041800406,2019-06-13,TOR @ GSW,W
2,42018,1610612744,GSW,0041800405,2019-06-10,GSW @ TOR,W
5,42018,1610612761,TOR,0041800404,2019-06-07,TOR @ GSW,W
6,42018,1610612761,TOR,0041800403,2019-06-05,TOR @ GSW,W
9,42018,1610612744,GSW,0041800402,2019-06-02,GSW @ TOR,W
...,...,...,...,...,...,...,...
4160,12018,15021,BJD,0011800004,2018-09-29,BJD @ DAL,L
4161,12018,1610612750,MIN,0011800005,2018-09-29,MIN @ GSW,W
4163,12018,104,PER,0011800006,2018-09-29,PER @ UTA,L
4167,12018,15016,MEL,0011800001,2018-09-28,MEL @ PHI,L


## Filling home/away team columns

In [6]:
def season_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        season_w = past_games['WL'].value_counts()['W']
        # season_l = past_games['WL'].value_counts()['L']
        # season_w = season_w / (season_w + season_l)
    except KeyError:
        season_w = 0
    return season_w

def last10_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last10_w = past_games['WL'].head(10).value_counts()['W']
        # last10_l = past_games['WL'].head(10).value_counts()['L']
        # last10_w = last10_w / (last10_w + last10_l)
    except KeyError:
        last10_w = 0
    return last10_w

def last5_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last5_w = past_games['WL'].head(5).value_counts()['W']
        # last5_l = past_games['WL'].head(5).value_counts()['L']
        # last5_w = last5_w / (last5_w + last5_l)
    except KeyError:
        last5_w = 0
    return last5_w

def season_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        season_w_opp = past_games[past_mathups]['WL'].value_counts()['W']
        # season_l_opp = past_games[past_mathups]['WL'].value_counts()['L']
        # season_w_opp = season_w_opp / (season_w_opp + season_l_opp)
    except KeyError:
        season_w_opp = 0
    return season_w_opp

def last10_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last10_w_opp = past_games[past_mathups]['WL'].head(10).value_counts()['W']
        # last10_l_opp = past_games[past_mathups]['WL'].head(10).value_counts()['L']
        # last10_w_opp = last10_w_opp / (last10_w_opp + last10_l_opp)
    except KeyError:
        last10_w_opp = 0
    return last10_w_opp

def last5_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last5_w_opp = past_games[past_mathups]['WL'].head(5).value_counts()['W']
        # last5_l_opp = past_games[past_mathups]['WL'].head(5).value_counts()['L']
        # last5_w_opp = last5_w_opp / (last5_w_opp + last5_l_opp)
    except KeyError:
        last5_w_opp = 0
    return last5_w_opp

def avg_pts(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_pts = past_games.head(20)['PTS'].mean()
    return avg_pts

def avg_fg_pct(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_fg_pct = past_games.head(20)['FG_PCT'].mean()
    return avg_fg_pct

def avg_fga(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_fga = past_games.head(20)['FGA'].mean()
    return avg_fga

def avg_fg3m(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_fg3m = past_games.head(20)['FG3M'].mean()
    return avg_fg3m

def avg_ftm(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_ftm = past_games.head(20)['FTM'].mean()
    return avg_ftm

def avg_dreb(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_dreb = past_games.head(20)['DREB'].mean()
    return avg_dreb

def avg_oreb(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_oreb = past_games.head(20)['OREB'].mean()
    return avg_oreb

def avg_reb(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_reb = past_games.head(20)['REB'].mean()
    return avg_reb

def avg_stl(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_stl = past_games.head(20)['STL'].mean()
    return avg_stl

home_team_data.rename(columns={'H_TEAM': 'TEAM'}, inplace=True)
home_team_data["H_SSN_W"] = home_team_data.apply(season_w, axis=1)
home_team_data["H_LAST10_W"] = home_team_data.apply(last10_w, axis=1)
home_team_data["H_LAST5_W"] = home_team_data.apply(last5_w, axis=1)
home_team_data["H_SEASON_W_OPP"] = home_team_data.apply(season_w_opp, axis=1)
home_team_data["H_LAST10_W_OPP"] = home_team_data.apply(last10_w_opp, axis=1)
home_team_data["H_LAST5_W_OPP"] = home_team_data.apply(last5_w_opp, axis=1)
home_team_data["H_AVG_PTS"] = home_team_data.apply(avg_pts, axis=1)
home_team_data["H_AVG_FG_PCT"] = home_team_data.apply(avg_fg_pct, axis=1)
home_team_data["H_AVG_FGA"] = home_team_data.apply(avg_fga, axis=1)
home_team_data["H_AVG_FG3M"] = home_team_data.apply(avg_fg3m, axis=1)
home_team_data["H_AVG_FTM"] = home_team_data.apply(avg_ftm, axis=1)
home_team_data["H_AVG_DREB"] = home_team_data.apply(avg_dreb, axis=1)
home_team_data["H_AVG_OREB"] = home_team_data.apply(avg_oreb, axis=1)
home_team_data["H_AVG_REB"] = home_team_data.apply(avg_reb, axis=1)
home_team_data["H_AVG_STL"] = home_team_data.apply(avg_stl, axis=1)
home_team_data.rename(columns={'TEAM': 'H_TEAM'}, inplace=True)

away_team_data.rename(columns={'A_TEAM': 'TEAM'}, inplace=True)
away_team_data["A_SSN_W"] = away_team_data.apply(season_w, axis=1)
away_team_data["A_LAST10_W"] = away_team_data.apply(last10_w, axis=1)
away_team_data["A_LAST5_W"] = away_team_data.apply(last5_w, axis=1)
away_team_data["A_SSN_W_OPP"] = away_team_data.apply(season_w_opp, axis=1)
away_team_data["A_LAST10_W_OPP"] = away_team_data.apply(last10_w_opp, axis=1)
away_team_data["A_LAST5_W_OPP"] = away_team_data.apply(last5_w_opp, axis=1)
away_team_data["A_AVG_PTS"] = away_team_data.apply(avg_pts, axis=1)
away_team_data["A_AVG_FG_PCT"] = away_team_data.apply(avg_fg_pct, axis=1)
away_team_data["A_AVG_FGA"] = away_team_data.apply(avg_fga, axis=1)
away_team_data["A_AVG_FG3M"] = away_team_data.apply(avg_fg3m, axis=1)
away_team_data["A_AVG_FTM"] = away_team_data.apply(avg_ftm, axis=1)
away_team_data["A_AVG_DREB"] = away_team_data.apply(avg_dreb, axis=1)
away_team_data["A_AVG_OREB"] = away_team_data.apply(avg_oreb, axis=1)
away_team_data["A_AVG_REB"] = away_team_data.apply(avg_reb, axis=1)
away_team_data["A_AVG_STL"] = away_team_data.apply(avg_stl, axis=1)
away_team_data.rename(columns={'TEAM': 'A_TEAM'}, inplace=True)

home_team_data

,SEASON_ID,TEAM_ID,H_TEAM,GAME_ID,GAME_DATE,MATCHUP,H_TEAM_WIN,H_SSN_W,H_LAST10_W,H_LAST5_W,...,H_LAST5_W_OPP,H_AVG_PTS,H_AVG_FG_PCT,H_AVG_FGA,H_AVG_FG3M,H_AVG_FTM,H_AVG_DREB,H_AVG_OREB,H_AVG_REB,H_AVG_STL
0,42018,1610612744,GSW,0041800406,2019-06-13,GSW vs. TOR,L,72,7,2,...,2,114.00,0.47660,86.15,12.15,19.85,32.85,10.00,42.85,7.25
3,42018,1610612761,TOR,0041800405,2019-06-10,TOR vs. GSW,L,77,7,4,...,4,106.35,0.44340,84.60,12.10,19.75,34.20,8.15,42.35,8.25
4,42018,1610612744,GSW,0041800404,2019-06-07,GSW vs. TOR,L,71,7,3,...,1,116.00,0.48045,87.35,12.10,20.10,33.20,10.35,43.55,7.30
7,42018,1610612744,GSW,0041800403,2019-06-05,GSW vs. TOR,L,71,7,4,...,1,116.15,0.48480,87.25,12.10,19.55,34.15,10.25,44.40,7.30
8,42018,1610612761,TOR,0041800402,2019-06-02,TOR vs. GSW,L,75,7,5,...,3,106.35,0.45070,84.65,12.15,18.30,35.25,7.75,43.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,12018,1610612744,GSW,0011800005,2018-09-29,GSW vs. MIN,L,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4164,12018,1610612761,TOR,0011800003,2018-09-29,TOR vs. POR,W,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4165,12018,1610612742,DAL,0011800004,2018-09-29,DAL vs. BJD,W,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4166,12018,1610612755,PHI,0011800001,2018-09-28,PHI vs. MEL,W,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
away_team_data

,SEASON_ID,TEAM_ID,A_TEAM,GAME_ID,GAME_DATE,MATCHUP,WL,A_SSN_W,A_LAST10_W,A_LAST5_W,...,A_LAST5_W_OPP,A_AVG_PTS,A_AVG_FG_PCT,A_AVG_FGA,A_AVG_FG3M,A_AVG_FTM,A_AVG_DREB,A_AVG_OREB,A_AVG_REB,A_AVG_STL
1,42021,1610612744,GSW,0042100406,2022-06-16,GSW @ BOS,W,73,7,3,...,3,111.75,0.48400,86.05,13.65,15.20,34.50,9.55,44.05,7.60
3,42021,1610612738,BOS,0042100405,2022-06-13,BOS @ GSW,L,67,6,3,...,3,106.00,0.44765,82.15,14.15,18.35,34.35,8.80,43.15,6.25
4,42021,1610612744,GSW,0042100404,2022-06-10,GSW @ BOS,W,71,6,2,...,2,113.75,0.49515,85.25,14.20,15.55,34.05,9.40,43.45,7.15
6,42021,1610612744,GSW,0042100403,2022-06-08,GSW @ BOS,L,71,7,3,...,2,113.75,0.49240,85.40,13.85,16.20,34.70,9.75,44.45,7.20
8,42021,1610612738,BOS,0042100402,2022-06-05,BOS @ GSW,L,66,7,4,...,2,109.35,0.46160,82.60,14.05,19.00,34.35,8.95,43.30,6.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,12021,1610612753,ORL,0012100003,2021-10-04,ORL @ BOS,L,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,12021,1610612755,PHI,0012100002,2021-10-04,PHI @ TOR,L,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4178,12021,1610612756,PHX,0012100009,2021-10-04,PHX @ SAC,L,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4179,12021,1610612766,CHA,0012100006,2021-10-04,CHA @ OKC,W,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## combining home and away team data

In [7]:
# drop useless columns
home_team_data.drop(columns=['SEASON_ID', 'TEAM_ID', 'GAME_DATE', 'MATCHUP'], inplace=True)
away_team_data.drop(columns=['SEASON_ID', 'TEAM_ID', 'GAME_DATE', 'MATCHUP', 'WL'], inplace=True)


In [8]:
# merge home and away data around game id
game_data = pd.merge(home_team_data, away_team_data, left_on='GAME_ID', right_on='GAME_ID')
# remove game id row
game_data.drop(columns=['GAME_ID'], inplace=True)

#replace NaNs with 0
game_data = game_data.fillna(0)
game_data.to_csv('game_data.csv')
game_data

,H_TEAM,H_TEAM_WIN,H_SSN_W,H_LAST10_W,H_LAST5_W,H_SEASON_W_OPP,H_LAST10_W_OPP,H_LAST5_W_OPP,H_AVG_PTS,H_AVG_FG_PCT,...,A_LAST5_W_OPP,A_AVG_PTS,A_AVG_FG_PCT,A_AVG_FGA,A_AVG_FG3M,A_AVG_FTM,A_AVG_DREB,A_AVG_OREB,A_AVG_REB,A_AVG_STL
0,GSW,L,72,7,2,2,2,2,114.00,0.47660,...,3,106.70,0.44335,84.50,11.95,20.35,33.95,8.40,42.35,8.10
1,TOR,L,77,7,4,5,5,4,106.35,0.44340,...,1,114.75,0.47790,86.65,11.85,20.20,33.40,10.30,43.70,7.45
2,GSW,L,71,7,3,1,1,1,116.00,0.48045,...,4,106.65,0.44660,84.65,12.15,19.40,34.40,8.10,42.50,8.15
3,GSW,L,71,7,4,1,1,1,116.15,0.48480,...,3,105.55,0.44315,84.95,11.90,18.85,34.60,8.15,42.75,7.90
4,TOR,L,75,7,5,3,3,3,106.35,0.45070,...,0,117.25,0.48895,88.00,12.05,19.20,34.20,10.50,44.70,7.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,GSW,L,0,0,0,0,0,0,0.00,0.00000,...,0,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2081,TOR,W,0,0,0,0,0,0,0.00,0.00000,...,0,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2082,DAL,W,0,0,0,0,0,0,0.00,0.00000,...,0,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2083,PHI,W,0,0,0,0,0,0,0.00,0.00000,...,0,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Collecting Advanced Box score data

In [ ]:
boxscore_data = pd.read_csv('adv_bosxscore3.csv')

def net_rating(row):
    last10_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])].head(10)
    sum = 0
    for i in range(len(last10_games)):
        relevant_game = boxscore_data[(boxscore_data['gameId'] == last10_games.iloc[i]['GAME_ID']) & (boxscore_data['teamId'] == row['TEAM_ID'])]
        print(len(relevant_game))
        sum = sum + relevant_game['offensiveRating'] - relevant_game['defensiveRating']
        # print(type(sum))
    return sum / 10

home_team_data["NET_RATING"] = home_team_data.apply(net_rating, axis=1)

In [7]:
from nba_api.stats.endpoints import boxscoreadvancedv3
advanced_boxscore3 = pd.DataFrame()

def advanced_boxscore_applier(row, advanced_boxscore3):
    try:
        advanced_boxscore = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=row['GAME_ID'])
        advanced_boxscore = advanced_boxscore.get_data_frames()[1]
        advanced_boxscore3 = pd.concat([advanced_boxscore3, advanced_boxscore])
        return advanced_boxscore3
    except TimeoutError:
        return advanced_boxscore3
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return advanced_boxscore3

for i in range(2100, 2159):
    advanced_boxscore3 = advanced_boxscore_applier(home_team_data.iloc[i], advanced_boxscore3)
advanced_boxscore3.to_csv('2100-2149_adv_bosxscore3.csv')
advanced_boxscore3

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,0012200057,1610612747,Los Angeles,Lakers,LAL,lakers,240:00,98.0,99.1,104.2,...,17.5,0.506,0.569,1.0,0.200,114.24,114.0,95.00,114.0,0.490
1,0012200057,1610612750,Minnesota,Timberwolves,MIN,timberwolves,240:00,104.2,103.5,98.0,...,10.5,0.495,0.535,1.0,0.200,114.24,114.0,95.00,114.0,0.510
0,0012200054,1610612755,Philadelphia,76ers,PHI,sixers,240:00,96.4,98.0,91.3,...,11.9,0.455,0.502,1.0,0.199,102.80,101.0,84.17,101.0,0.510
1,0012200054,1610612766,Charlotte,Hornets,CHA,hornets,240:00,91.3,93.1,96.4,...,23.8,0.525,0.541,1.0,0.198,102.80,101.0,84.17,101.0,0.490
0,0012200053,1610612754,Indiana,Pacers,IND,pacers,240:00,105.3,105.8,93.7,...,16.5,0.529,0.564,1.0,0.199,105.12,103.0,85.83,103.0,0.588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0012200003,1610612763,Memphis,Grizzlies,MEM,grizzlies,240:00,105.1,105.9,100.8,...,21.8,0.558,0.596,1.0,0.197,101.50,101.5,84.58,101.0,0.576
0,0012200002,1610612746,LA,Clippers,LAC,clippers,240:00,115.3,117.5,78.9,...,23.3,0.591,0.631,1.0,0.198,103.78,104.0,86.67,103.0,0.796
1,0012200002,50009,Ra'anana,Maccabi Ra'anana,MRA,maccabi-ra'anana,240:00,78.9,77.1,115.3,...,16.2,0.347,0.411,1.0,0.197,103.78,104.0,86.67,105.0,0.204
0,0012200001,1610612764,Washington,Wizards,WAS,wizards,240:00,83.5,82.9,92.7,...,13.3,0.405,0.448,1.0,0.200,103.90,105.5,87.92,105.0,0.441


In [ ]:
endpoints import boxscoreadvancedv3
advanced_boxscore3 = pd.DataFrame()

def advanced_boxscore_applier(row, advanced_boxscore3):
    try:
        advanced_boxscore = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=row['GAME_ID'])
        advanced_boxscore = advanced_boxscore.get_data_frames()[1]
        advanced_boxscore3 = pd.concat([advanced_boxscore3, advanced_boxscore])
        return advanced_boxscore3
    except TimeoutError:
        return advanced_boxscore3
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return advanced_boxscore3

for i in range(2100, 2159):
    advanced_boxscore3 = advanced_boxscore_applier(home_team_data.iloc[i], advanced_boxscore3)
advanced_boxscore3.to_csv('2100-2149_adv_bosxscore3.csv')
advanced_boxscore3

In [9]:
csv_files = ['0-999_adv_bosxscore3.csv', '1000-2150_adv_bosxscore3.csv']
dataframes = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_csv('adv_bosxscore3.csv')

In [7]:
games_2024 = leaguegamefinder.LeagueGameFinder(season_nullable='2024-25')
games_2024 = games_2024.get_data_frames()[0]
games_2024

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
